In [12]:
import numpy as np
import pandas as pd; pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt

# utility
import time
import copy as cp
from tqdm import tqdm
from collections import defaultdict

# 乱数
rng = np.random.RandomState(1234)
random_state = 42



# pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from torchvision import datasets, transforms, models

/Users/keimy/anaconda3/envs/world_models/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/keimy/anaconda3/envs/world_models/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <5AA8DD3D-A2CC-31CA-8060-88B4E9C18B09> /Users/keimy/anaconda3/envs/world_models/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <F9D8FD58-F19A-3314-BC55-A8367346C2A9> /Users/keimy/anaconda3/envs/world_models/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [14]:
conda env export > environment.yml



Note: you may need to restart the kernel to use updated packages.
